In [1]:
import nglview as nv

import json
from pymatgen.core import Lattice, Structure, Molecule

In [2]:
def read_pymatgen_dict(file):
    with open(file, "r") as f:
        d = json.load(f)
    return Structure.from_dict(d)

In [3]:
file = '../Data/dichalcogenides_public/structures/6141cf11cc0e69a0cf28ab35.json'
crystal = read_pymatgen_dict(file)

In [4]:
view = nv.show_pymatgen(crystal)
# view.add_unitcell()
view

NGLWidget()

In [5]:
nv.write_html('embed.html', view)

In [6]:
coords = [[0, 0, 0], [0.75,0.5,0.75]]
lattice = Lattice.from_parameters(a=3.84, b=3.84, c=3.84, alpha=120,
                                  beta=90, gamma=60)
struct = Structure(lattice, ["Si", "Si"], coords)

coords = [[0.000000, 0.000000, 0.000000],
          [0.000000, 0.000000, 1.089000],
          [1.026719, 0.000000, -0.363000],
          [-0.513360, -0.889165, -0.363000],
          [-0.513360, 0.889165, -0.363000]]
methane = Molecule(["C", "H", "H", "H", "H"], coords)

nv.show_pymatgen(methane)

NGLWidget()